In [1]:
from bs4 import BeautifulSoup
import requests, datetime, re, html

NameSearch=re.compile(r'^[A-Z \-,\.\(\)`“”\d!&/’]+(\(via (tele)?phone\))?(\(subtitles\))?(\(voice-over\))?(\(through translator\))?[:;]')
TimeSearch=re.compile(r'^\d\d?:\d\d')
DescriptionSearch=re.compile(r'^\([A-Z “”,\d]+\)?[\)0]$')
verbose=0

In [2]:
last_name_tran={'AKEMAN':'AKERMAN','WATER':'WATERS','SANDER':'SANDERS','SANERS':'SANDERS','SANDES':'SANDERS',
'MORE':'MOORE','MOORES':'MOORE','BLUMEHTHAL':'BLUMENTHAL','BLUMETHAL':'BLUMENTHAL','WILSOM':'WILSON','RODGER':'RODGERS',
'MOOORE':'MOORE','REINDER':'REINER','SWWET':'SWEET','SWEE':'SWEET','BRON':'BROWN','MICCONNELL':'MCCONNELL',
'AYEAR':'AYER','COBBB':'COBB','COBBS':'COBB','COATSE':'COATES','GOLDBER':'GOLDBERG','GERCETTI':'GARCETTI',
'HAYSE':'HAYES','HAYEWS':'HAYES','HNAYES':'HAYES','HAYEW':'HAYES','HAYEDS':'HAYES','HAYESS':'HAYES','HEYAS':'HAYES',
'HAYAES':'HAYES','HAYESA':'HAYES','HAYESW':'HAYES','HAYHES':'HAYES','HAYW':'HAYES','HGAYES':'HAYES','HAYDES':'HAYES',
'HJAYES':'HAYES','HYAES':'HAYES','HAEYS':'HAYES','HAES':'HAYES','HYES':'HAYES','YAES':'HAYES','HATES':'HAYES','AYES':'HAYES',
'HAYS':'HAYES','HAYE':'HAYES','HAEYES':'HAYES','SHAYES':'HAYES','HAYEES':'HAYES',
'MADDOWE':'MADDOW','MADDDOW':'MADDOW','MADDSOW':'MADDOW','MADWO':'MADDOW','MADSOW':'MADDOW','MADOW':'MADDOW',
'VELSH':'VELSHI','VELNSHI':'VELSHI',
'KORNAKCI':'KORNACKI','KOTNACKI':'KORNACKI',
'JENTLESEON':'JENTLESON','ISKIKOFF':'ISIKOFF','ISKOFF':'ISIKOFF','JOENS':'JONES','RTAN':'RYAN','CHIRSTIE':'CHRISTIE',
'TRUM':'TRUMP','TRUMPS':'TRUMP','TRUMP)':'TRUMP','TRUMPP':'TRUMP','TUMP':'TRUMP','TRMP':'TRUMP','TURMP':'TRUMP',
'TRUMO':'TRUMP','TRUYMP':'TRUMP','TRIMP':'TRUMP',
'SHHA':'SHAH','OCACIO-CORTEZ':'OCASIO-CORTEZ','OABAMA':'OABAMA','SHUGARMEN':'SHUGARMAN','SHEHEED':'SHAHID','ROSELLO':'ROSSELLO',
'SHWARTZ':'SCHWARTZ','SUSTEIN':'SUNSTEIN','SHERROD':'BROWN','HANNITTY':'HANNITY','PIEREE-LOUIS':'PIERRE-LOUIS','OBAM':'OBAMA',
'EICNER':'EICHNER','EDWARDSS':'EDWARDS','EARNETS':'EARNEST','DIESNT':'DIENST','COBELLA':'COBIELLA','ROACH':'ROCAH',
'JENTLESEON':'JENTLESON','MSYTAL':'MYSTAL','HINOSA':'HINOJOSA','HINJOSA':'HINOJOSA','BARLETT':'BARTLETT',
'FELKINS':'FILKINS','CAOTES':'COATES','WILSOIN':'WILSON','WEBERT':'WEBER','LUKIANNOFF':'LUKIANOFF','TEACHOT':'TEACHOUT',
'ARREDOINDO':'ARREDONDO','PACHECHO':'PACHECO','GIGNEY':'GIBNEY','O`RIEN':'O`BRIEN','SANCROCK':'SANDROCK',
'CANCELLA':'CANCELA','JARDINS':'JARDIN','DILLINGER':'DELLINGER','KARA-MURSA':'KARA-MURZA','O`NEIL':'O`NEILL',
'CARSLWELL':'CARSWELL','MNUNCHIN':'MNUCHIN','BANNO':'BANNON','PENNYPIECE':'PETTYPIECE','PRUIT':'PRUITT',
'MCONNELL':'MCCONNELL','MCCONELL':'MCCONNELL','MCCONNEL':'MCCONNELL','MCCINNELL':'MCCONNELL',
'AWRD':'WARD','BARRA':'BARRO','FITZERALD':'FITZGERALD','HETZBERG':'HERTZBERG','OBEDLLAH':'OBEDALLAH','PENNYPIERCE':'PENNYPIECE',
'HOTS':'MELBER','WISON':'WILSON','FAMILGLIETTI':'FAMIGLIETTI','MEELBER':'MELBER','DIAZ-BALARRT':'DIAZ-BALART',
'JELTESEN':'JELTSEN','CARIFIN':'CARIFI','PAMIERI':'PALMIERI','FAROW':'FARROW','MORRISEEY':'MORRISSEY','CLAREK':'CLARKE',
'MILL':'MOODIE-MILLS','MILEER':'MILLER','MERKELY':'MERKLEY','MERKELEY':'MERKLEY','MELER':'MELBER','MELMBER':'MELBER',
'APRIL':'RYAN','MUPHY':'MURPHY',
'MATHEWS':'MATTHEWS','MATHTEWS':'MATTHEWS','MATTEHWS':'MATTHEWS','MATTEWS':'MATTHEWS','MATTHES':'MATTHEWS',
'MATTHESWS':'MATTHEWS','MATTHEWES':'MATTHEWS','MATTHEW':'MATTHEWS','MATTHEWSD':'MATTHEWS','MATTHWES':'MATTHEWS',
'MATTTHEWS':'MATTHEWS','MATTWES':'MATTHEWS','MTTEHWS':'MATTHEWS','ATTHEWS':'MATTHEWS',
'CAPHEHART':'CAPEHART','FIENMAN':'FINEMAN','ELLISION':'ELLISON','(MENENDEZ':'MENENDEZ','MNAXWELL':'MAXWELL',
'COHNE':'COHEN','MURRY':'MURRAY','MURAY':'MURRAY','PRYZYBLA':'PRZYBYLA','TIDD':'TODD','TOOD':'TODD',
'MADDOWW':'MADDOW','KUSNERH':'KUSHNER','FREDERICKON':'FREDRICKSON','SCHIMDT':'SCHMIDT','KO9RNACKI':'KORNACKI',
'CROWLEYT':'CROWLEY','STEEEL':'STEEL','SCAMARUCCI':'SCARAMUCCI','REPORER':'REPORTER','GUILIAN':'GUILIANI','GIULIANA':'GUILIANI',
'PELOS':'PELOSI','O’REILLY':'O`REILLY','DILIANIAN':'DILANIAN','DILANIANI':'DILANIAN',
'LIUE':'LIEU','YAMICHE':'ALCINDOR','GOLBERG':'GOLDBERG',
'BOELHERT':'BOEHLERT','BORNSTEIN,':'BORNSTEIN','CAFIN':'CHAFIN','CAMPTELL':'RAMPELL','WILISON':'WILSON',
'WILLAMS':'WILLIAMS','WIILIAMS':'WILLIAMS','WILIAMS':'WILLIAMS','WILLAIMS':'WILLIAMS','WILLIAM':'WILLIAMS','WILLIAMNS':'WILLIAMS',
'WILLIANS':'WILLIAMS','WILLIAS':'WILLIAMS','WILLIMAS':'WILLIAMS','WILLIMS':'WILLIAMS','WLLIAMS':'WILLIAMS','WILILAMS':'WILLIAMS',
'MAREY':'MARKEY','CUNNINGHUM':'CUNNINGHAM','MCCASKIL':'MCCASKILL','MCCASSKILL':'MCCASKILL','EHOMAS':'THOMAS','GOODWN':'GOODWIN',
'MARKEY0':'MARKEY','MARKAYE':'MARKAY','MARSHAL':'MARSHALL','MARHALL':'MARSHALL','MONTANA':'TESTER','MALBER':'MELBER',
'WATERES':'WATERS','STEELES':'STEELE','STELEE':'STEELE','VLESHI':'VELSHI','KORANCKI':'KORNACKI','SERNWER':'SERWER',
'KORANACKI':'KORNACKI','SACARAMUCCI':'SCARAMUCCI','KAINES':'KAINE','SCHFF':'SCHIFF','MCMULLEN':'MCMULLIN','RIED':'REID',
'CHRSITIE':'CHRISTIE','WILKENSON':'WILKERSON','CUMMING':'CUMMINGS','COOMEY':'COMEY','SWALWEWLL':'SWALWELL','TROT':'TROTT',
'GUITERREZ':'GUTIERREZ','GONZELZ':'GONZALEZ','GIRDHARADAS':'GHIRDHARADAS','GIRIDHARARDAS':'GHIRDHARADAS',
'SADDIQUI':'SIDDIQUI','BROWNE-DIANIS':'DIANIS','CARPENTETR':'CARPENTER','SHCIFF':'SCHIFF','WLLACE':'WALLACE',
'KOOBUCHAR':'KLOBUCHAR','KLOBUCHER':'KLOBUCHAR','FIGLIUZZ':'FIGLIUZZI','FIGLIUZI':'FIGLIUZZI',                
'GIANFORTT':'GIANFORTE','GETIERREZ':'GUTIERREZ','ANNUONCER':'ANNOUNCER','HUELBKAMP':'HUELSKAMP','ANNNOUNCER':'ANNOUNCER',
'GESSER':'GESSEN','GERNER':'GERTNER','GERANHOLM':'GRANHOLM','GARGIELD':'GARFIELD','FLOREDS':'FLORES','FLEGENHEIMERL':'FLEGENHEIMER',
'SHCUMER':'SCHUMER','BANKS':'WINE-BANKS','BANK':'WINE-BANKS','SSANDERS':'SANDERS','LAWRENCE':'WILKERSON','NICHOLES':'NICHOLS',
'SESSION':'SESSIONS','RUBEN':'RUBIN','O`CONELL':'O`CONNELL',
'O`DONELL':'O`DONNELL','O`DONNEL':'O`DONNELL','O`SONNELL':'O`DONNELL','O”DONNELL':'O`DONNELL','O’DONNEL':'O`DONNELL',
'O’DONNELL':'O`DONNELL','O`DONNE':'O`DONNELL',
'SEDEN':'SEDER','HARIS':'HARRIS','O`BREIN':'O`BRIEN','KAYTAL':'KATYAL','THIMAS':'THOMAS',
'GOLDBERBG':'GOLDBERG','GOLBERGE':'GOLDBERG','BARXOER':'BOXER','BASSINS':'BASSIN','BASIN':'BASSIN','BOEHLTER':'BOEHLERT',
'SANDFORD':'SANFORD','(COPPINS)':'COPPINS','(CROSSTALK)NUZZI':'NUZZI','“MILLER”':'MILLER','"MILLER"':'MILLER','BLUENTHAL':'BLUMENTHAL',
'REYNODS':'REYNOLDS','RAMPBELL':'RAMPELL','PIERC':'PIERCE','PIECE':'PIERCE','PERCE':'PIERCE','PEIRCE':'PIERCE','WITES':'WITTES',
'FERENHOLD':'FARENTHOLD','CLLIMACHI':'CALLIMACHI','BARRAGN':'BARRAGAN','EARNERS':'EARNEST','WOORUFF':'WOODRUFF',
'PODEST':'PODESTA','BETO':'O`ROURKE','RASIN':'RASKIN','PICCOLINI':'PICCIOLINI','HARDWOOD':'HARWOOD','FORES':'FLORES',
'ENTON':'ENTEN','CORETES':'CORTES','ALCINDO':'ALCINDOR','AMBRAMS':'ABRAMS','BANCI':'BANCO','BRICE':'BRYCE','FEGIE':'FEIGE',
'KORNAKCKI':'KORNACKI','SCHIF':'SCHIFF','SCHIEFF':'SCHIFF','GREEWALD':'GREENWALD','KHRUSHCEVA':'KHRUSHCHEVA',
'DELANDY':'DILANIAN','DOMENENCH':'DOMENECH','DOWELL':'DOWDELL','HARRI':'HARRIS','NGYUEN':'NGUYEN','NALDER':'NADLER',
'BURGES':'BURGESS','CARDON':'CARDIN','CARMONE':'CARMON','CLAR':'CLARK','CONSASON':'CONASON','DEGETTTE':'DEGETTE','ADELE':'ADKINS',
'SCHMIT':'SCHMIDT','SCHNEIDERAMN':'SCHNEIDERMAN','OABAMA':'OBAMA','LEHTINEN':'ROS-LEHTINEN','ISGUR':'FLORES',
'SERGEAT':'SERGEANT','STIGLTIZ':'STIGLITZ','VAQUERIO':'VAQUEIRO','ELMERS':'ELLMERS','GOWDEN':'GOWDY',
'SOLTIS-ANDRESON':'SOLTIS-ANDERSON','STANGIO':'STRANGIO','SYKERS':'SYKES','COASTES':'COATES','BERADI':'BERARDI',
'WEISMASS':'WEISMANN','WALWELL':'SWALWELL','WELBER':'MELBER','VAUSE':'VELSHI','TURN':'TURNER','SARADON':'SARANDON',
'JONSTON':'JOHNSTON','JAELOUS':'JEALOUS','IOFEE':'IOFFE','HSOT':'KORNACKI','HOLTAMANN':'HOLTZMAN','HAMILLL':'HAMILL',
'MCHEE':'MCGHEE','WALLCE':'WALLACE','RATNNER':'RATTNER','KUSNER':'KUSHNER','KRK':'KIRK','KINKIAD':'KINKAID','KELIN':'KLEIN',               
'MOODY-MILLE':'MOODIE-MILLS','MOODY-MILLS':'MOODIE-MILLS','HAYEDEN':'HAYDEN','MACCARTHY':'MCCARTHY','SEINFED':'SEINFELD',
'LITTMAN':'LITMAN','GIULINI':'GIULIANI','GUILLIANI':'GIULIANI','GIULIANIA':'GIULIANI','GIULIAN':'GIULIANI',
'BAILER':'BAILEY','CHUCH':'CHURCH',
'SHAPRTON':'SHARPTON','KRISTOFF':'KRISTOF',"VANDENHEUVEL":"VANDEN HEUVEL"}
full_names={'MOORE':'MICHAEL MOORE',"HAYES":"CHRIS HAYES",'TRUMP':"DONALD TRUMP",'STEELE':"MICHAEL STEELE",
 'SCHULKIN':'DAVID SCHULKIN','SEDER':'SAM SEDER','SHAHID':'WALEED SHAHID','STRANGIO':'CHASE STRANGIO','ROSENSTEIN':'ROD ROSENSTEIN',
 "MATTHEWS":"CHRIS MATTHEWS","FIORINA":"CARLY FIORINA",'SESSIONS':'JEFF SESSIONS','RUBIN':'JENNIFER RUBIN','BOEHLERT':'ERIC BOEHLERT',
 "MADDOW":"RACHEL MADDOW","KORNACKI":"STEVE KORNACKI","WAGNER":"ALEX WAGNER","MELBER":"ARI MELBER","MCCONNELL":"MITCH MCCONNELL",
 "PENCE":"MIKE PENCE","BIDEN":"JOE BIDEN","KHANA":"RO KHANA","ZUCKERBERG":"MARK ZUCKERBERG","SNOWDEN":"EDWARD SNOWDEN",
'DUNN-WRIGHT':'RHIANA DUNN-WRIGHT','COOK-HAWKINS':'APRIL COOK-HAWKINS','GIULIANI':'RUDY GIULIANI','HIFFE':'JUDY HIFFE',
'CHAFIN':'TRUMAN CHAFIN','HOLT':'LESTER HOLT','HASSELBECK':'ELISABETH HASSELBECK','BOLLER':'ERIC BOLLER',
'HATHROUSI':'SAMIA HATHROUSI','BELAFONTE':'HARRY BELAFONTE','CAPEHART':'JONATHAN CAPEHART',
 "ISIKOFF":"MICHAEL ISIKOFF","KRISTOF":"NICHOLAS KRISTOF","VANDEN HEUVEL":"KATRINA VANDEN HEUVEL","STEPHANOPOULOS":"GEORGE STEPHANOPOULOS",
 "RUBIO":"MARCO RUBIO","COTTON":"TOM COTTON","KAINE":"TIM KAINE","MCMULLIN":"EVAN MCMULLIN","PELOSI":"NANCY PELOSI",'CORN':'DAVID CORN',
 "OBAMA":"PRESIDENT BARACK OBAMA","VANCE":"JOYCE VANCE","CHRISTIE":"CHRIS CHRISTIE",'ESPAILLAT':'ADRIANO ESPAILLAT','JENTLESON':'ADAM JENTLESON',
'PANZA':'LUCY PANZA','BOEHNER':'JOHN BOEHNER','KIRIAKOU':'JOHN KIRIAKOU','BUTTIGIEG':'PETE BUTTIGIEG','LENNIG':'CAROL LENNIG',
'PIRRO':'JEANINE PIRRO','GHIRDHARADAS':'ANAND GHIRDHARADAS','ELLMERS':'RENEE ELLMERS','GORKOV':'SERGEI GORKOV','ADKINS':'ADELE ADKINS',
'NADLER':'JERRY NADLER','BOYANG':'MIEKE BOYANG','PENNYPIECE':'SHANNON PENNYPIECE','BOXER':'BARBARA BOXER','O`ROURKE':'BETO O`ROURKE',
 'DILANIAN':'KEN DILANIAN','WILEY':'MAYA WILEY','WITTES':'BEN WITTES','WOODRUFF':'BETSY WOODRUFF','SANTORUM':'RICK SANTORUM',
 'WILKERSON':"LAWRENCE WILKERSON","COMEY":"JAMES COMEY","DEVOS":"BETSY DEVOS","SCHUMER":"CHUCK SHUMER","WINE-BANKS":"JILL WINE-BANKS",
 "BOEHNER":"JOHN BOEHNER","NANCE":"MALCOLM NANCE","HANNITY":"SEAN HANNITY","SPICER":"SEAN SPICER",'MNUCHIN':'STEVE MNUCHIN',
 "LITMAN":"HARRY LITMAN","NUNES":"DEVIN NUNES","HIRONO":"MAZIE HIRONO","GUILIANI":"RUDY GUILIANI"}
single_word_names={'PROTESTERS','PROTESTER','PRINCE','CROWD','INTERVIEWER',"REPORTER","HOST","REPUBLICAN","OFFICER",
"QUESTIONER","FEMALE",'PROSECUTOR','ACTIVIST','SPORTSCASTER','SUPPORTERS',
 "QUESTION","UNKNOWN","(UNKNOWN)","OPERATOR","AUDIENCE","BOY","GIRL","CALLER","METEOROLOGIST",'TOURE',
 "DEMONSTRATORS","UNIDENTIFIED","ANNOUNCER","MODERATOR",'CHILD','CHILDREN','STAFF','JAY-Z','ELMO','ANU','GIRLS','NARRATOR',
'AKON','REPUBLICANS','PUPPET','NYT','MOTHER','HECKLER','POLICE','DRIVER','DISPATCHER','911','EMPLOYEE','JUDGE','FOREMAN'}
special_URL=['MSNBC-LIVE-WITH-ARI-MELBER/2020-03-30']

In [69]:
import importlib
import sys
importlib.reload(sys.modules['Transcript_Show_Functions'])
from Transcript_Show_Functions import *
Shows=['THE BEAT WITH ARI MELBER','MSNBC LIVE','THE 11TH HOUR WITH BRIAN WILLIAMS','THE LAST WORD','RACHEL MADDOW SHOW','HARDBALL','ALL IN']
Fix_Function = {Show:Show.replace(' ','_')+'_Fix' for Show in Shows}
Last_Name_Fix_Function = {Show:Show.replace(' ','_')+'_Last_Name_Fix' for Show in Shows}
Fix_Speaker_Function = {Show:Show.replace(' ','_')+'_Fix_Speakers' for Show in Shows}

In [76]:
def Get_Name(Show,text,Speakers,Guests,AiredOn,problem_names,last_name_tran,full_names,single_word_names,Last_Name_Fix_Function):
    speaker=text.lstrip()
    comma=speaker.find(', ')
    if comma!=-1:
        if (speaker[comma+2:comma+5]=='SR.')| \
            (speaker[comma+2:comma+5]=='JR.'):
            speaker=speaker[:comma+5]
            space=speaker[:comma].rfind(' ')
            if space!=-1:
                last_name=speaker[space+1:comma]
                if last_name in last_name_tran:
                    last_name=last_name_tran[last_name]
                Speakers[last_name]=speaker
            else:
                last_name=speaker
        else:
            speaker=speaker[:comma]
            paren=speaker.find(' (')
            if paren!=-1:
                speaker=speaker[:paren]
            if speaker[-3:] in ['SR.','JR.','III'] or speaker[-2:] in ['IV','II']:
                last_name=speaker[:speaker.rfind(' ')]
                last_name=last_name[last_name.rfind(' ')+1:]
                if last_name in last_name_tran:
                    last_name=last_name_tran[last_name]
                Speakers[last_name]=speaker
            else:
                space=speaker.rfind(' ')
                if space!=-1:
                    last_name=speaker[space+1:]
                    if last_name in last_name_tran:
                        last_name=last_name_tran[last_name]
                    Speakers[last_name]=speaker
                else:
                    last_name=speaker
    else:
        paren=speaker.find(' (')
        if paren!=-1:
            speaker=speaker[:paren]
        speaker=speaker.strip()
        space=speaker.rfind(' ')
        if space!=-1:
            last_name=speaker[space+1:]
            if last_name in last_name_tran:
                last_name=last_name_tran[last_name]
            Speakers[last_name]=speaker.strip()
        else:
            last_name=speaker

    # Process last names if they don't quite fit in Speakers list
    if last_name in last_name_tran:
        last_name=last_name_tran[last_name]
    else:
        last_name=eval(Last_Name_Fix_Function[Show])(last_name,AiredOn)

    if last_name not in Speakers:
        if last_name in Guests:
            Speakers[last_name]=Guests[last_name]
        else:
            Speakers,nochange = eval(Fix_Speaker_Function[Show])(last_name,AiredOn,Speakers)
            if nochange:
                if last_name in full_names:
                    Speakers[last_name]=full_names[last_name]
                elif last_name in single_word_names:
                    Speakers[last_name]=last_name
                else:
                    if last_name not in problem_names:
                        problem_names[last_name]=[AiredOn,text]
    if last_name in Speakers:
        cs=Speakers[last_name]
    else:
        cs='NONE:'+last_name
    return Speakers,cs,problem_names

In [93]:
def Output_Data(current_speaker,first_paragraph,SQLConn,Show,paragraph_string,Speakers,Guests,AiredOn,problem_names):
    nooutput=False
    paragraph_string=paragraph_string.lstrip()
    paragraph_string=eval(Fix_Function[Show])(paragraph_string.replace(chr(160),chr(32)),AiredOn)
    findname = NameSearch.search(paragraph_string)
    findtime = TimeSearch.search(paragraph_string)
    if findname is not None \
            and findtime is None \
            and (findname.span()[1]>5 or paragraph_string[findname.span()[1]-1]!=';'):
        endingloc = findname.span()[1]-1
        if paragraph_string[endingloc] not in [':',';']:
            endingloc-=1
        Speakers,current_speaker,problem_names=Get_Name(Show,paragraph_string[:endingloc],Speakers,Guests,AiredOn,problem_names,last_name_tran,full_names,single_word_names,Last_Name_Fix_Function)
        assert current_speaker,'0No current_speaker on '+AiredOn.strftime('%Y-%m-%d %H:%M:%S')+' for '+paragraph_string+' for show: '+Show
        paragraph_string=paragraph_string[endingloc+2:]
    elif first_paragraph:
        nooutput=True
        current_speaker=''
    else:
        assert current_speaker,'1No current_speaker on '+AiredOn.strftime('%Y-%m-%d %H:%M:%S')+' for '+paragraph_string+' for show: '+Show
    if not nooutput and paragraph_string: #'No text on '+AiredOn.strftime('%Y-%m-%d %H:%M:%S')+' for '+current_speaker
        SQL="Exec Add_Transcript_Text '"+Show+"','"+AiredOn.strftime('%Y-%m-%d %H:%M:%S')+"','"+current_speaker.replace("'","''")+"','"+paragraph_string.replace("'","''")+"'"
        try:
            cursorconn=SQLConn.cursor()
            cursorconn.execute(SQL)
            cursorconn.commit() 
            cursorconn.close()
        except pyodbc.Error as e:
            print("Exec Add_Transcript_Text '"+Show+"','"+AiredOn.strftime('%Y-%m-%d %H:%M:%S')+"','"+current_speaker.replace("'","''")+"','"+paragraph_string.replace("'","''")+"'")
            print(str(e))
    return Speakers,current_speaker,problem_names

In [88]:
from bs4 import NavigableString,Tag
def check_url():
    Show='THE BEAT WITH ARI MELBER'
    Speakers,problem_names=dict(),dict()
    if Show=='THE 11TH HOUR WITH BRIAN WILLIAMS':
        URL='http://www.msnbc.com/transcripts/11TH-HOUR-WITH-BRIAN-WILLIAMS'+'/2019-02-22'
    if Show=='THE BEAT WITH ARI MELBER':
        URL='http://www.msnbc.com/transcripts/MSNBC-LIVE-WITH-ARI-MELBER'+'/2020-03-30'
    else:
        URL='http://www.msnbc.com/transcripts/'+Show.replace(' ','-')+'/2020-03-30'
    if Show=='ALL IN':
        title='All In with Chris Hayes Transcripts | MSNBC'
    elif Show=='HARDBALL':
        title='Hardball with Chris Matthews Transcripts | MSNBC' 
    elif Show=='RACHEL MADDOW SHOW':
        title='The Rachel Maddow Show Transcripts | MSNBC' 
    elif Show=='THE LAST WORD':
        title="The Last Word with Lawrence O'Donnell Transcripts | MSNBC" 
    elif Show=='THE 11TH HOUR WITH BRIAN WILLIAMS':
        title='The 11th Hour with Brian Williams Transcripts | MSNBC'
    elif Show=='MSNBC LIVE':
        title='MSNBC Live Transcripts | MSNBC'
    elif Show=='THE BEAT WITH ARI MELBER':
        title='The Beat with Ari Melber Transcripts | MSNBC'
    try:
        rawpage=requests.get(URL)
        rawpage.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)
        return False
    page=BeautifulSoup(html.unescape(rawpage.text))
    # Generic page when there is no transcript for a specific date URL
    if page.find('title').text==title:
        print('bad url',URL)
        return False
    main_section = page.find_all(class_='field')
    if len(main_section)!=3:
        print(URL,'missing section')
        return False
    else:
        AiredOn=datetime.datetime.strptime(main_section[0].text.strip(),'%m/%d/%y %I:%M %p')
        Guests={guest.strip()[guest.strip().rfind(' ')+1:].upper():guest.strip().upper() for guest in main_section[1].text.strip().split(',')}

    first_paragraph=True
    problem_names,Speakers,current_speaker,paragraph_string=dict(),dict(),None,''
    Done=False
    for paragraph in main_section[2].find_all('p'):
        if paragraph.contents:
            #print('|',paragraph.contents,'|',sep='')
            lineidx=0
            while lineidx < len(paragraph.contents):
                #print(lineidx,first_paragraph)
                if isinstance(paragraph.contents[lineidx],Tag) and paragraph.contents[lineidx].name=='br':
                    lineidx+=1
                    if lineidx<len(paragraph.contents):#'No unexpected ending of <p> tag like 3 <br> in a row
                        if isinstance(paragraph.contents[lineidx],Tag) and paragraph_string \
                                and paragraph.contents[lineidx].name=='br': #Two successive <br> means end of paragraph
                            #print(-1,paragraph_string)
                            Speakers,current_speaker,problem_names=Output_Data(current_speaker,first_paragraph,SQLConn,Show,paragraph_string,Speakers,Guests,AiredOn,problem_names)
                            #print(0,current_speaker,paragraph_string)
                            if first_paragraph:
                                first_paragraph=False
                            paragraph_string=''
                            lineidx+=1
                else:
                    if isinstance(paragraph.contents[lineidx],NavigableString):
                        if (paragraph.contents[lineidx].string.strip()==u'\xa0' and first_paragraph and lineidx==0 and not paragraph_string):
                            first_paragraph=False
                            line=''
                        elif (paragraph.contents[lineidx].string.strip()==u'\xa0' and paragraph_string): #end of paragraph
                            #print(-2,paragraph_string,sep='')
                            Speakers,current_speaker,problem_names=Output_Data(current_speaker,first_paragraph,SQLConn,Show,paragraph_string,Speakers,Guests,AiredOn,problem_names)
                            #print(1,current_speaker,paragraph_string)
                            line=''
                            paragraph_string=''
                            if first_paragraph:
                                #print(1)
                                first_paragraph=False
                        else:    
                            line=paragraph.contents[lineidx].string.strip()
                        if first_paragraph and line.startswith('Show: '):
                            ShowName=line[6:]
                            if (Show == 'ALL IN' and ShowName.upper()!='ALL IN WITH CHRIS HAYES') \
                                or (Show == 'HARDBALL' and ShowName.upper() not in ['HARDBALL','HARDBALL WITH CHRIS MATTHEWS']) \
                                or (Show == 'MSNBC LIVE' and ShowName.upper()!='MSNBC LIVE') \
                                or (Show == 'THE BEAT WITH ARI MELBER' and ShowName.upper()!='THE BEAT WITH ARI MELBER') \
                                or (Show == 'THE LAST WORD' and ShowName.upper() not in ['THE LAST WORD WITH LAWRENCE O`DONNELL','The Last Word with Lawrence O’Donnell','THE LAST WORD WITH LAWRENCE O’DONNELL']) \
                                or (Show == '11TH HOUR WITH BRIAN WILLIAMS' and ShowName.upper()!='THE 11TH HOUR WITH BRIAN WILLIAMS') \
                                or (Show == 'RACHEL MADDDOW SHOW' and ShowName.upper() != 'THE RACHEL MADDOW SHOW'):
                                print(Show,AiredOn,ShowName)
                        if first_paragraph and (line.startswith('Show: ') or line.startswith('Date: ') or line.startswith('Guest: ')):
                            line=''
                    elif isinstance(paragraph.contents[lineidx],Tag) and paragraph.contents[lineidx].text.strip():
                        line=paragraph.contents[lineidx].text.strip()
                    else:
                        line=''

                    if line == 'END' or line.startswith('THIS IS A RUSH TRANSCRIPT.'): # end of transcript signal
                        Done=True                                                                                            
                        break
                    elif DescriptionSearch.search(line) is not None:
                        lineidx+=3
                        paragraph_string=''
                    else:
                        if line:
                            paragraph_string+=(' ' if paragraph_string else '')+html.unescape(line).replace(u'\xa0','')
                        lineidx+=1
                #print(line)
            if URL[33:] in special_URL and paragraph_string: #NameSearch.search(paragraph_string) is not None:  #unusual case of paragraph ending
            #    #print(current_speaker,paragraph_string)
                Speakers,current_speaker,problem_names=Output_Data(current_speaker,first_paragraph,SQLConn,Show,paragraph_string,Speakers,Guests,AiredOn,problem_names)
                #print(2,current_speaker,paragraph_string)
                paragraph_string=''
                if first_paragraph:
                    first_paragraph=False
        if Done:
            break
    if len(problem_names)>0:
        print(Show,problem_names)
        return False
    else:
        return True    

#print(check_url())

In [94]:
def Process_MSNBC_URL(URL,Show):
    if Show=='ALL IN':
        title='All In with Chris Hayes Transcripts | MSNBC'
    elif Show=='HARDBALL':
        title='Hardball with Chris Matthews Transcripts | MSNBC' 
    elif Show=='RACHEL MADDOW SHOW':
        title='The Rachel Maddow Show Transcripts | MSNBC' 
    elif Show=='THE LAST WORD':
        title="The Last Word with Lawrence O'Donnell Transcripts | MSNBC" 
    elif Show=='THE 11TH HOUR WITH BRIAN WILLIAMS':
        title='The 11th Hour with Brian Williams Transcripts | MSNBC'
    elif Show=='MSNBC LIVE':
        title='MSNBC Live Transcripts | MSNBC'
    elif Show=='THE BEAT WITH ARI MELBER':
        title='The Beat with Ari Melber Transcripts | MSNBC'
    try:
        rawpage=requests.get(URL)
        rawpage.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)
        return False
    page=BeautifulSoup(html.unescape(rawpage.text))
    # Generic page when there is no transcript for a specific date URL
    if page.find('title').text==title:
        return False
    main_section = page.find_all(class_='field')
    if len(main_section)!=3:
        print(URL,'missing section')
        return False
    else:
        AiredOn=datetime.datetime.strptime(main_section[0].text.strip(),'%m/%d/%y %I:%M %p')
        Guests={guest.strip()[guest.strip().rfind(' ')+1:].upper():guest.strip().upper() for guest in main_section[1].text.strip().split(',')}
    first_paragraph=True
    problem_names,Speakers,current_speaker,paragraph_string=dict(),dict(),None,''
    Done=False
    for paragraph in main_section[2].find_all('p'):
        if paragraph.contents:
            lineidx=0
            while lineidx < len(paragraph.contents):
                if isinstance(paragraph.contents[lineidx],Tag) and paragraph.contents[lineidx].name=='br':
                    lineidx+=1
                    if lineidx<len(paragraph.contents):#'No unexpected ending of <p> tag like 3 <br> in a row
                        if isinstance(paragraph.contents[lineidx],Tag) and paragraph_string and paragraph.contents[lineidx].name=='br': #Two successive <br> means end of paragraph
                            Speakers,current_speaker,problem_names=Output_Data(current_speaker,first_paragraph,SQLConn,Show,paragraph_string,Speakers,Guests,AiredOn,problem_names)
                            #print(0,URL,current_speaker,sep='|')
                            if first_paragraph:
                                first_paragraph=False
                            paragraph_string=''
                            lineidx+=1
                else:
                    if isinstance(paragraph.contents[lineidx],NavigableString):
                        if (paragraph.contents[lineidx].string.strip()==u'\xa0' and first_paragraph and lineidx==0 and not paragraph_string):
                            first_paragraph=False
                            line=''
                        elif (paragraph.contents[lineidx].string.strip()==u'\xa0' and paragraph_string): #end of paragraph
                            Speakers,current_speaker,problem_names=Output_Data(current_speaker,first_paragraph,SQLConn,Show,paragraph_string,Speakers,Guests,AiredOn,problem_names)
                            #print(1,URL,current_speaker,sep='|')
                            line=''
                            paragraph_string=''
                            if first_paragraph:
                                first_paragraph=False
                        else:    
                            line=paragraph.contents[lineidx].string.strip()
                        if first_paragraph and line.startswith('Show: '):
                            ShowName=line[6:]
                            if (Show == 'ALL IN' and ShowName.upper()!='ALL IN WITH CHRIS HAYES') \
                                or (Show == 'HARDBALL' and ShowName.upper() \
                                    not in ['HARDBALL','HARDBALL WITH CHRIS MATTHEWS','Hardball with Chris Matthews g']) \
                                or (Show == 'MSNBC LIVE' and ShowName.upper()!='MSNBC LIVE') \
                                or (Show == 'THE BEAT WITH ARI MELBER' and ShowName.upper()!='THE BEAT WITH ARI MELBER') \
                                or (Show == 'THE LAST WORD' and ShowName.upper() \
                                    not in ['THE LAST WORD WITH LAWRENCE O`DONNELL','The Last Word with Lawrence O’Donnell','THE LAST WORD WITH LAWRENCE O’DONNELL']) \
                                or (Show == '11TH HOUR WITH BRIAN WILLIAMS' and ShowName.upper()!='THE 11TH HOUR WITH BRIAN WILLIAMS') \
                                or (Show == 'RACHEL MADDDOW SHOW' and ShowName.upper() != 'THE RACHEL MADDOW SHOW'):
                                print(Show,AiredOn,ShowName)
                        if first_paragraph and (line.startswith('Show: ') or line.startswith('Date: ') or line.startswith('Guest: ')):
                            line=''
                    elif isinstance(paragraph.contents[lineidx],Tag) and paragraph.contents[lineidx].text.strip():
                        line=paragraph.contents[lineidx].text.strip()
                    else:
                        line=''

                    if line == 'END' or line.startswith('THIS IS A RUSH TRANSCRIPT.'): # end of transcript signal
                        Done=True                                                                                            
                        break
                    elif DescriptionSearch.search(line) is not None:
                        lineidx+=3
                        paragraph_string=''
                    else:
                        if line:
                            paragraph_string+=(' ' if paragraph_string else '')+html.unescape(line).replace(u'\xa0','')
                        lineidx+=1
            if URL[33:] in special_URL and paragraph_string:
                Speakers,current_speaker,problem_names=Output_Data(current_speaker,first_paragraph,SQLConn,Show,paragraph_string,Speakers,Guests,AiredOn,problem_names)
                paragraph_string=''
                if first_paragraph:
                    first_paragraph=False                        
        if Done:
            break
    if len(problem_names)>0:
        print(Show,problem_names)

In [95]:
import pyodbc
pyodbc.pooling = False
SQLConn = pyodbc.connect("Driver=SQL Server;Server=(local);Database=MSM;Trusted_Connection=true")

In [96]:
Done=False
for year in range(2019, 2020):
    for month in range(12):
        for day in range(31):
            for Show in Shows:
                Date_Is_Valid=True
                try:
                    URL_Date=datetime.date(year,month+1,day+1)
                except ValueError:
                    Date_Is_Valid=False
                if Date_Is_Valid and URL_Date.weekday()<=4:
                    if URL_Date > datetime.date.today()-datetime.timedelta(days=2 if datetime.date.today().weekday()<=2 else 4):
                        Done=True
                    else:
                        if Show=='THE 11TH HOUR WITH BRIAN WILLIAMS':
                            URL=Show[4:]
                        elif Show=='THE BEAT WITH ARI MELBER':
                            URL='MSNBC LIVE '+Show[9:]
                        else:
                            URL=Show
                        URL='http://www.msnbc.com/transcripts/'+URL.replace(' ','-')+'/'+str(year)+'-'+str(month+1).zfill(2)+'-'+str(day+1).zfill(2)
                        Process_MSNBC_URL(URL,Show)
                if Done:
                    break
            if Done:
                break
        if Done:
            break
    if Done:
        break

In [97]:
SQLConn.close()

In [ ]:
def test_name(Show,AiredOn,text):
    text=globals()[Show.replace(' ','_')+'_Fix'](text.replace(chr(160),chr(32)),AiredOn)                    
    findtext = NameSearch.search(text)
    assert findtext is not None,'regex problem on name'
    print(text[:findtext.span()[1]-1])
    Speakers,cur_speaker,problem_names=Get_Name(Show,text[:findtext.span()[1]-1],dict(),dict(),AiredOn,dict())
    print(Speakers,cur_speaker)

In [ ]:
#test_name('HARDBALL',datetime.datetime(2015,3,18,0,0,0),'JESSICA STERN, AUTHOR, “ISIS, The STATE OF TERROR”: I think so. I')